# **Homework 2: Phoneme Classification**


Objectives:
* Solve a classification problem with deep neural networks (DNNs).
* Understand recursive neural networks (RNNs).

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Download Data
Download data from google drive, then unzip it.

You should have
- `libriphone/train_split.txt`: training metadata
- `libriphone/train_labels`: training labels
- `libriphone/test_split.txt`: testing metadata
- `libriphone/feat/train/*.pt`: training feature
- `libriphone/feat/test/*.pt`:  testing feature

after running the following block.

> **Notes: if the google drive link is dead, you can download the data directly from [Kaggle](https://www.kaggle.com/c/ml2023spring-hw2/data) and upload it to the workspace.**


In [1]:
!pip install --upgrade gdown

# Main link
# !gdown --id '1N1eVIDe9hKM5uiNRGmifBlwSDGiVXPJe' --output libriphone.zip
!gdown --id '1qzCRnywKh30mTbWUEjXuNT2isOCAPdO1' --output libriphone.zip

!unzip -q libriphone.zip
!ls libriphone

'gdown' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'unzip' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'ls' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


# Some Utility Functions
**Fixes random number generator seeds for reproducibility.**

In [10]:
import numpy as np
import torch
import random

def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

**Helper functions to pre-process the training data from raw MFCC features of each utterance.**

A phoneme may span several frames and is dependent to past and future frames. \
Hence we concatenate neighboring phonemes for training to achieve higher accuracy. The **concat_feat** function concatenates past and future k frames (total 2k+1 = n frames), and we predict the center frame.

Feel free to modify the data preprocess functions, but **do not drop any frame** (if you modify the functions, remember to check that the number of frames are the same as mentioned in the slides)

In [11]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

#     return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)
    return x.permute(1, 0, 2).view(seq_len, concat_n, feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    
    # Data Preparation:
    # The code starts by defining the number of classes (class_num) for the classification task, which is set to 41.
    class_num = 41 # NOTE: pre-computed, should not need change

    # Mode Selection:
    # The variable mode is determined based on the split argument. If the split is 'train' or 'val', the mode is set to 'train'; otherwise, it's set to 'test'.
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    # Label Dictionary:
    # If the mode is not 'test', the code loads phone labels from a file named {mode}_labels.txt in the phone_path directory and stores them in a dictionary called label_dict. 
    # Each entry in the dictionary consists of a filename as the key and a list of integers (presumably class labels) as the value.
    label_dict = {}
    if mode != 'test':
        phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

        for line in phone_file:
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]

    # Data Splitting:

    # If the split is 'train' or 'val', the code reads a file named 'train_split.txt' and 'test_split.txt' from the phone_path directory, 
    # and randomly shuffles the list of filenames (usage list). It then splits this list into training and validation sets based on the train_ratio. 
    # The usage list is used to identify the data files to be processed for the chosen split.
    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')
    
    
    # Data Loading and Concatenation:

    # The code iterates through the usage list and, for each filename, does the following:
    
    # Loads a feature vector (feat) from a file named {fname}.pt in the feat_dir/mode directory.
    
    # Checks the length of the feature vector (cur_len).
    
    # Concatenates concat_nframes consecutive frames of the feature vector (feat) together. The feature vector dimensions appear to be (frames, 39) before concatenation.
    
    # If the mode is not 'test', it loads the corresponding label for the filename from the label_dict.
    
    # Copies the feature vectors and labels into a larger tensor (X for features and y for labels). The idx variable is used to keep track of the current position in the tensor.
    
    # Updates the idx to account for the number of feature vectors processed.
    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    X = torch.empty((max_len, concat_nframes, 39))
    if mode != 'test':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes) # contate number of frame into 1 vector
        if mode != 'test':
            label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :, :] = feat
        if mode != 'test':
            y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :, :]
    if mode != 'test':
        y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
        print(y.shape)
        return X, y
    else:
        return X

# Dataset

In [12]:
import torch
from torch.utils.data import Dataset

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


# Model
Feel free to modify the structure of the model.

## LSTM

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
    
    def __init__(self, dropout=0.1):
        super(Classifier, self).__init__()
        self.lstm_1 = nn.LSTM(39, 256, batch_first=True, bidirectional=True, dropout=dropout)
        self.ln_1   = nn.LayerNorm(512)
        self.lstm_2 = nn.LSTM(512, 128, batch_first=True, bidirectional=True, dropout=dropout)
        self.ln_2   = nn.LayerNorm(256)
        self.lstm_3 = nn.LSTM(256, 64, batch_first=True, bidirectional=True, dropout=dropout)
        self.linear = nn.Linear(128, 41)

    def forward(self, x):
        lstm_1_output, _ = self.lstm_1(x)
        lstm_2_output, _ = self.lstm_2(self.ln_1(lstm_1_output))
        lstm_3_output, (ht, ct) = self.lstm_3(self.ln_2(lstm_2_output))
        ht = torch.cat((ht[0], ht[1]), 1)  # 把 BiLSTM 的兩層結果接起來
        ht = ht.unsqueeze(0)  # 添加第0維
        linear_output = self.linear(ht[-1])
        return linear_output

# Hyper-parameters

In [14]:
# data prarameters
# TODO: change the value of "concat_nframes" for medium baseline
concat_nframes = 25   # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.8   # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 1213          # random seed
batch_size = 256        # batch size
num_epoch = 10         # the number of training epoch
learning_rate = 0.005      # learning rate
model_path = './model.ckpt'   # the path where the checkpoint will be saved

# model parameters
# TODO: change the value of "hidden_layers" or "hidden_dim" for medium baseline
input_dim = 39 * concat_nframes  # the input dim of the model, you should not change the value
# hidden_layers = 2         # the number of hidden layers
# hidden_dim = 64           # the hidden dim

# Dataloader

In [8]:
torch.cuda.is_available()

True

In [7]:
from torch.utils.data import DataLoader
import gc

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# torch size
# [Batch Size (Batch Dimension), Sequence Length (Time Steps 週期)), Hidden Size (Features) ] 

[Dataset] - # phone classes: 41, number of utterances for train: 2743


2743it [00:10, 262.54it/s]


[INFO] train set
torch.Size([1696055, 25, 39])
torch.Size([1696055])
[Dataset] - # phone classes: 41, number of utterances for val: 686


686it [00:03, 211.55it/s]


[INFO] val set
torch.Size([420739, 25, 39])
torch.Size([420739])


In [15]:
import numpy as np

same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cuda


# Training

In [16]:
# create model, define a loss function, and optimizer
# model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)

# create model, define a loss function, and optimizer
model = Classifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()

    # validation
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader)):
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)
            outputs = model(features)

            loss = criterion(outputs, labels)

            _, val_pred = torch.max(outputs, 1)
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
            val_loss += loss.item()

    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

    # if the model improves, save a checkpoint at this epoch
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f'saving model with acc {best_acc/len(val_set):.5f}')


c:\Users\anson\Documents\GitHub\LearnDL\.venv\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
100%|██████████| 1644/1644 [00:27<00:00, 60.85it/s]


[001/010] Train Acc: 0.59426 Loss: 1.35091 | Val Acc: 0.66287 loss: 1.10578
saving model with acc 0.66287


100%|██████████| 1644/1644 [00:27<00:00, 59.79it/s]


[002/010] Train Acc: 0.70243 Loss: 0.95878 | Val Acc: 0.69418 loss: 0.98897
saving model with acc 0.69418


100%|██████████| 1644/1644 [00:28<00:00, 57.23it/s]


...

[008/010] Train Acc: 0.79196 Loss: 0.64783 | Val Acc: 0.73918 loss: 0.86262
saving model with acc 0.73918


100%|██████████| 1644/1644 [00:17<00:00, 92.62it/s]


[009/010] Train Acc: 0.79757 Loss: 0.62810 | Val Acc: 0.73898 loss: 0.86424


100%|██████████| 1644/1644 [00:17<00:00, 94.24it/s]

[010/010] Train Acc: 0.80312 Loss: 0.60735 | Val Acc: 0.73614 loss: 0.88019


In [17]:
del train_set, val_set
del train_loader, val_loader
gc.collect()

49

# Testing
Create a testing dataset, and load model from the saved checkpoint.

In [18]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

[Dataset] - # phone classes: 41, number of utterances for test: 857


857it [00:10, 79.13it/s]

[INFO] test set
torch.Size([527364, 25, 39])


In [20]:
# load model
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [21]:
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)


100%|██████████| 2061/2061 [00:13<00:00, 156.07it/s]


Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [22]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))